Import necessary libraries for Training the model

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, confusion_matrix)


Loading the preprocessed data 

In [ ]:
df=pd.read_csv('diabetic_data_cleaned.csv')

Preparing the data for training

In [ ]:
df['target'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

In [ ]:
#Drops unnecessary columns
cols_to_drop = ['encounter_id', 'patient_nbr', 'readmitted', 'weight', 'payer_code', 'medical_specialty']
df = df.drop(cols_to_drop, axis=1)

In [ ]:
#converts categorical variables to numerical
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

Training the model using Train-Test split of (70-30)

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"Class distribution in training set: \n{y_train.value_counts(normalize=True)}")